In [25]:
import numpy as np

from rembg import remove
from PIL import Image
from skimage import feature

In [39]:
# Đọc dữ liệu ảnh. 
in_img = Image.open("spaghetti.jpg")
# Xoá phần nền.
out_img = remove(in_img)

In [21]:
# in_img.show()
# out_img.show()

In [41]:
# Tạo nền trắng, ghép phần vật thể với phần nền mới
white_bg = Image.new("RGBA", size=in_img.size, color=(255, 255, 255))
white_bg.paste(out_img, (0, 0), mask=out_img)

In [ ]:
# white_bg.show()

In [28]:
# Lấy 3 kênh màu r, g, b riêng biệt
wr = white_bg.getdata(band=0)
wg = white_bg.getdata(band=1)
wb = white_bg.getdata(band=2)

In [29]:
# Xác định biên bằng canny
gray_img = white_bg.convert("L")
matrix = np.array(gray_img.getdata()).reshape(gray_img.size)
edge_canny = feature.canny(matrix, low_threshold=10, high_threshold=30)
height, width = white_bg.size

In [32]:
# Tính số lượng pixel nằm ở khu vực nền
co = 0
for i in range(height):
    if not any(edge_canny[i]):
        co += width
        continue
    limit = 0
    for j in range(width):
        if edge_canny[i][j] == True:
            limit = j
            break
        else:
            co += 1
    for j in range(width-1, limit, -1):
        if edge_canny[i][j] == True:
            # print(i, j)
            break
        else:
            co += 1
print(co)
print(height * width)

142255
266388


In [9]:
# Xây dựng biểu đồ tần suất cho 3 kênh màu của vật thể
r_his = wr.histogram()
g_his = wg.histogram()
b_his = wb.histogram()
bg_pixel = min(co, r_his[255], g_his[255], b_his[255])
r_his[255] = r_his[255] - bg_pixel
g_his[255] = g_his[255] - bg_pixel
b_his[255] = b_his[255] - bg_pixel

In [11]:
# Hàm tính biểu đồ tần suất tích luỹ
def acc_his(arr):
    new_arr = []
    s = sum(arr)
    v = 0
    for i in range(len(arr)):
        v += arr[i]
        new_arr.append(v*100/s)
    return new_arr

In [35]:
# Xây dựng đặc trưng cho ảnh
feature = []
feature += acc_his(r_his)
feature += acc_his(g_his)
feature += acc_his(b_his)
print(len(feature))

768
